# Lab 11: Grid Localization using Bayes Filter (Virtual Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

<hr>


In [1]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
from localization_extras import Localization

# Setup Logger
LOG = get_logger('demo_notebook.log')

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

gui.show()

# Start the simulator
START_SIM()

# Start the plotter
START_PLOTTER()

2024-04-28 16:04:54,945 | INFO     |: Logger demo_notebook.log initialized.


TwoByTwoLayout(children=(Label(value='Simulator', layout=Layout(grid_area='top-left', width='80px')), HBox(chi…

Loading Flatland...
Initializing pygame framework...


qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=614.357,633.614 gbl=614.357,633.614 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-614.357,-633.614 last=-614.357,-633.614 Δ 614.357,633.614) : no target window


2024-04-28 16:19:48,187 | INFO     |: Simulator is stopped
2024-04-28 16:19:48,797 | INFO     |: Plotter is stopped


In [2]:
# Initialize Robot to communicate with the virtual robot and plotter
robot = VirtualRobot(cmdr)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a VirtualRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

2024-04-28 16:07:46,997 | INFO     |:  | Number of observations per grid cell: 18
2024-04-28 16:07:46,998 | INFO     |:  | Precaching Views...


/Users/harrypeng/Desktop/ECE4160/FastRobots-sim-release-release-mirror/localization.py:150: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(distance_intersections_tt), intersections_tt[np.nanargmin(distance_intersections_tt)]


2024-04-28 16:07:48,188 | INFO     |:  | Precaching Time: 1.189 secs
2024-04-28 16:07:48,188 | INFO     |: Initializing beliefs with a Uniform Distribution
2024-04-28 16:07:48,189 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=1037.14,195.703 gbl=1037.14,195.703 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-1037.14,-195.703 last=-1037.14,-195.703 Δ 1037.14,195.703) : no target window


# Run the Bayes Filter
The cells below utilizes the member functions of class **Localization** (defined in [localization_extras.py](../localization_extras.py)) in each iteration of the Bayes filter algorithm to localize the robot in the grid map. <br>

In [3]:
# Reset Robot and Plots
robot.reset()
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.print_update_stats(plot_data=True)

# Plot Odom and GT
current_odom, current_gt = robot.get_pose()
cmdr.plot_gt(current_gt[0], current_gt[1])
cmdr.plot_odom(current_odom[0], current_odom[1])

2024-04-28 16:07:57,936 | INFO     |: Initializing beliefs with a Uniform Distribution
2024-04-28 16:07:57,937 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
2024-04-28 16:08:01,024 | INFO     |: Update Step
2024-04-28 16:08:01,026 | INFO     |:      | Update Time: 0.001 secs
2024-04-28 16:08:01,027 | INFO     |: ---------- UPDATE STATS -----------
2024-04-28 16:08:01,037 | INFO     |: GT index      : (6, 4, 9)
2024-04-28 16:08:01,037 | INFO     |: Bel index     : (5, 4, 9) with prob = 0.8760402
2024-04-28 16:08:01,038 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2024-04-28 16:08:01,038 | INFO     |: GT            : (0.000, 0.000, 360.000)
2024-04-28 16:08:01,039 | INFO     |: Belief        : (0.000, 0.000, 10.000)
2024-04-28 16:08:01,039 | INFO     |: POS ERROR     : (-0.000, -0.000, 350.000)
2024-04-28 16:08:01,040 | INFO     |: ---------- UPDATE STATS -----------


qt.pointer.dispatch: skipping QEventPoint(id=4 ts=0 pos=0,0 scn=875.384,533.888 gbl=875.384,533.888 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-875.384,-533.888 last=-875.384,-533.888 Δ 875.384,533.888) : no target window


In [4]:
# Initialize the Trajectory object
traj = Trajectory(loc)

# Run through each motion steps
for t in range(0, traj.total_time_steps):
    print("\n\n-----------------", t, "-----------------")
    
    prev_odom, current_odom, prev_gt, current_gt = traj.execute_time_step(t)
        
    # Prediction Step
    loc.prediction_step(current_odom, prev_odom)
    loc.print_prediction_stats(plot_data=True)
    
    # Get Observation Data by executing a 360 degree rotation motion
    loc.get_observation_data()
    
    # Update Step
    loc.update_step()
    loc.print_update_stats(plot_data=True)

# Uncomment the below line to wait for keyboard input between each iteration.
#   input("Press Enter to Continue")
        
    print("-------------------------------------")



----------------- 0 -----------------
2024-04-28 16:08:07,394 | INFO     |: Prediction Step
2024-04-28 16:08:07,426 | INFO     |:  | Prediction Time: 0.031 secs
2024-04-28 16:08:07,427 | INFO     |: ---------- PREDICTION STATS -----------
2024-04-28 16:08:07,443 | INFO     |: GT index         : (6, 3, 7)
2024-04-28 16:08:07,444 | INFO     |: Prior Bel index  : (5, 3, 5) with prob = 0.1064646
2024-04-28 16:08:07,444 | INFO     |: POS ERROR        : (0.277, 0.220, 30.179)
2024-04-28 16:08:07,445 | INFO     |: ---------- PREDICTION STATS -----------
2024-04-28 16:08:10,376 | INFO     |: Update Step
2024-04-28 16:08:10,380 | INFO     |:      | Update Time: 0.003 secs
2024-04-28 16:08:10,381 | INFO     |: ---------- UPDATE STATS -----------
2024-04-28 16:08:10,388 | INFO     |: GT index      : (6, 3, 7)
2024-04-28 16:08:10,390 | INFO     |: Bel index     : (6, 4, 6) with prob = 0.9999999
2024-04-28 16:08:10,391 | INFO     |: Bel_bar prob at index = 2.9362237215911855e-11
2024-04-28 16:08:

qt.pointer.dispatch: skipping QEventPoint(id=4 ts=0 pos=0,0 scn=788.206,540.767 gbl=788.206,540.767 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-788.206,-540.767 last=-788.206,-540.767 Δ 788.206,540.767) : no target window


2024-04-28 16:08:28,775 | INFO     |: Prediction Step
2024-04-28 16:08:28,791 | INFO     |:  | Prediction Time: 0.016 secs
2024-04-28 16:08:28,791 | INFO     |: ---------- PREDICTION STATS -----------
2024-04-28 16:08:28,809 | INFO     |: GT index         : (8, 0, 8)
2024-04-28 16:08:28,809 | INFO     |: Prior Bel index  : (9, 1, 8) with prob = 0.0907337
2024-04-28 16:08:28,810 | INFO     |: POS ERROR        : (-0.436, -0.149, 1448.947)
2024-04-28 16:08:28,811 | INFO     |: ---------- PREDICTION STATS -----------
2024-04-28 16:08:31,769 | INFO     |: Update Step
2024-04-28 16:08:31,771 | INFO     |:      | Update Time: 0.001 secs
2024-04-28 16:08:31,771 | INFO     |: ---------- UPDATE STATS -----------
2024-04-28 16:08:31,778 | INFO     |: GT index      : (8, 0, 8)
2024-04-28 16:08:31,778 | INFO     |: Bel index     : (8, 1, 9) with prob = 1.0
2024-04-28 16:08:31,778 | INFO     |: Bel_bar prob at index = 0.01650801704781463
2024-04-28 16:08:31,779 | INFO     |: GT            : (0.783, 

qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=1279.56,566.043 gbl=1279.56,566.043 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-1279.56,-566.043 last=-1279.56,-566.043 Δ 1279.56,566.043) : no target window
Traceback (most recent call last):
  File "/Users/harrypeng/Desktop/ECE4160/FastRobots-sim-release-release-mirror/src/plotter.py", line 263, in keyPressEvent
    if event.key() == Qt.Key_Escape:
AttributeError: type object 'Qt' has no attribute 'Key_Escape'
Traceback (most recent call last):
  File "/Users/harrypeng/Desktop/ECE4160/FastRobots-sim-release-release-mirror/src/plotter.py", line 263, in keyPressEvent
    if event.key() == Qt.Key_Escape:
AttributeError: type object 'Qt' has no attribute 'Key_Escape'


2024-04-28 16:08:37,822 | INFO     |: Prediction Step
2024-04-28 16:08:37,839 | INFO     |:  | Prediction Time: 0.016 secs
2024-04-28 16:08:37,839 | INFO     |: ---------- PREDICTION STATS -----------
2024-04-28 16:08:37,857 | INFO     |: GT index         : (11, 1, 11)
2024-04-28 16:08:37,857 | INFO     |: Prior Bel index  : (11, 0, 10) with prob = 0.2143596
2024-04-28 16:08:37,858 | INFO     |: POS ERROR        : (-0.269, 0.314, 1818.334)
2024-04-28 16:08:37,858 | INFO     |: ---------- PREDICTION STATS -----------


...

2024-04-28 16:09:31.741 python[2639:48147] Warning: Window move completed without beginning


2024-04-28 16:09:33,088 | INFO     |: Prediction Step
2024-04-28 16:09:33,104 | INFO     |:  | Prediction Time: 0.016 secs
2024-04-28 16:09:33,105 | INFO     |: ---------- PREDICTION STATS -----------
2024-04-28 16:09:33,122 | INFO     |: GT index         : (3, 3, 0)
2024-04-28 16:09:33,123 | INFO     |: Prior Bel index  : (3, 2, 1) with prob = 0.0831619
2024-04-28 16:09:33,123 | INFO     |: POS ERROR        : (-0.203, 0.405, 5731.895)
2024-04-28 16:09:33,124 | INFO     |: ---------- PREDICTION STATS -----------


2024-04-28 16:09:33.493 python[2639:48147] Warning: Window move completed without beginning


2024-04-28 16:09:36,065 | INFO     |: Update Step
2024-04-28 16:09:36,068 | INFO     |:      | Update Time: 0.001 secs
2024-04-28 16:09:36,068 | INFO     |: ---------- UPDATE STATS -----------
2024-04-28 16:09:36,080 | INFO     |: GT index      : (3, 3, 0)
2024-04-28 16:09:36,080 | INFO     |: Bel index     : (2, 3, 0) with prob = 1.0
2024-04-28 16:09:36,081 | INFO     |: Bel_bar prob at index = 0.0021999103069629605
2024-04-28 16:09:36,081 | INFO     |: GT            : (-0.812, -0.205, 5942.086)
2024-04-28 16:09:36,082 | INFO     |: Belief        : (-0.914, -0.305, -170.000)
2024-04-28 16:09:36,082 | INFO     |: POS ERROR     : (0.102, 0.100, 6112.086)
2024-04-28 16:09:36,083 | INFO     |: ---------- UPDATE STATS -----------
-------------------------------------


qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=750.944,200.074 gbl=750.944,200.074 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-750.944,-200.074 last=-750.944,-200.074 Δ 750.944,200.074) : no target window
qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=761.936,608.522 gbl=761.936,608.522 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-761.936,-608.522 last=-761.936,-608.522 Δ 761.936,608.522) : no target window
Traceback (most recent call last):
  File "/Users/harrypeng/Desktop/ECE4160/FastRobots-sim-release-release-mirror/src/plotter.py", line 263, in keyPressEvent
    if event.key() == Qt.Key_Escape:
AttributeError: type object 'Qt' has no attribute 'Key_Escape'
Traceback (most recent call last):
  File "/Users/harrypeng/Desktop/ECE4160/FastRobots-sim-release-release-mirror/src/plotter.py", line 263, in keyPressEvent
    if event.key() == Qt.Key_Escape:
AttributeError: type object 'Qt' has no attribute 'Key_Escape'
qt.pointer.dispatch: skipping QEventPoint(id